In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(color_codes=True)
import missingno as msno
import os

In [4]:
%%time
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

# 데이터 경로 맞춰주세요!
# 혹시 코랩환경을 사용하신다면 왼쪽 폴더모양 아이콘을 눌러 "train_data.csv"를 드래그&드롭으로 업로드한 후 사용해주세요
DATA_PATH = '../../../input/data/train_data.csv'
DATA_PATH_T = '../../../input/data/test_data.csv'
df_train = pd.read_csv(DATA_PATH, dtype=dtype, parse_dates=['Timestamp'])
df_train = df_train.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)
df_test = pd.read_csv(DATA_PATH_T, dtype=dtype, parse_dates=['Timestamp'])
df_test = df_test.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

CPU times: user 4.39 s, sys: 456 ms, total: 4.85 s
Wall time: 4.86 s


In [5]:
data=pd.concat([df_train,df_test])
data.drop_duplicates(subset=["userID", "assessmentItemID"], keep="last", inplace=True)

train_all_data = data[data.answerCode >= 0]
test_data = data[data.answerCode < 0]

userid, itemid = (
    sorted(list(set(data.userID))),
    sorted(list(set(data.assessmentItemID))),
)
n_user, n_item = len(userid), len(itemid)

# userid_2index, itemid_2_index는 text로 저장할 파일임
userid_2_index = {v: i for i, v in enumerate(userid)}
itemid_2_index = {v: i for i, v in enumerate(itemid)}


In [6]:
a=df_test.groupby('userID').tail(1)

In [7]:
b=a.Timestamp.tolist()

In [8]:
test_noinference_data = df_test[df_test.answerCode >= 0]
test_valid_data=test_noinference_data.groupby('userID').tail(1)
test_noinference_data.drop(test_noinference_data.groupby('userID').tail(1).index,inplace=True)

/tmp/ipykernel_21997/2347239064.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_noinference_data.drop(test_noinference_data.groupby('userID').tail(1).index,inplace=True)


In [9]:
test_valid_data

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
1034,3,A050133007,A050000133,0,2020-10-26 13:13:11,5289
1705,4,A070146007,A070000146,1,2020-12-27 02:47:31,9080
3022,13,A070111007,A070000111,1,2020-12-27 04:35:01,9660
4282,17,A090064005,A090000064,1,2020-10-30 05:47:22,2611
4669,26,A060135006,A060000135,0,2020-10-23 11:44:01,1422
...,...,...,...,...,...,...
260051,7395,A040122004,A040000122,0,2020-09-08 02:05:18,2102
260066,7404,A030111004,A030000111,1,2020-10-13 09:47:31,7636
260081,7416,A050193003,A050000193,0,2020-10-04 02:44:17,10402
260096,7417,A050193003,A050000193,0,2020-09-06 13:08:54,10402


In [10]:
test_noinference_data

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623
...,...,...,...,...,...,...
260107,7439,A040197005,A040000197,0,2020-08-21 07:39:40,2132
260108,7439,A040197006,A040000197,1,2020-08-21 07:39:45,2132
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832


In [11]:
train_no_valid_inference_data=pd.concat([df_train,test_noinference_data])

# making validation data
valid_data_dic={}
for index, row in test_valid_data.iterrows():
    if row['userID'] in valid_data_dic:
        valid_data_dic[row['userID']].append(itemid_2_index[row['assessmentItemID']])
    else:
        valid_data_dic[row['userID']]=[itemid_2_index[row['assessmentItemID']]]

with open('valid.txt','w',encoding='UTF-8') as f:
    for code,name in valid_data_dic.items():
        f.write(f'{code}')
        for i in range(len(name)):
            f.write(f' {name[i]}')
        f.write(f'\n')



In [12]:
# making answercode
answerCode=test_valid_data['answerCode']
answerCode=answerCode.tolist()
print(len(answerCode))
answerCode=pd.DataFrame(answerCode)
answerCode.to_csv('answerCode.csv',index=False)

744


In [13]:
answerCode

,0
0,0
1,1
2,1
3,1
4,0
...,...
739,0
740,1
741,0
742,0


In [14]:
train_no_valid_inference_data=pd.concat([df_train,test_noinference_data])

# making train_no_valid_inference_data
neg_no_valid_inference=train_no_valid_inference_data[train_no_valid_inference_data.answerCode > 0]
train_no_valid_inference_data=train_no_valid_inference_data[train_no_valid_inference_data.answerCode == 0]
neg_no_valid_inference_dic={}
train_no_valid_inference_data_dic={}

for index, row in neg_no_valid_inference.iterrows():
    if row['userID'] in neg_no_valid_inference_dic:
        neg_no_valid_inference_dic[row['userID']].append(itemid_2_index[row['assessmentItemID']])
    else:
        neg_no_valid_inference_dic[row['userID']]=[itemid_2_index[row['assessmentItemID']]]
        
for index, row in train_no_valid_inference_data.iterrows():
    if row['userID'] in train_no_valid_inference_data_dic:
        train_no_valid_inference_data_dic[row['userID']].append(itemid_2_index[row['assessmentItemID']])
    else:
        train_no_valid_inference_data_dic[row['userID']]=[itemid_2_index[row['assessmentItemID']]]





In [15]:
train_no_valid_inference_data

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
6,0,A060003001,A060000003,0,2020-03-26 05:52:03,7226
18,0,A060005006,A060000005,0,2020-03-31 05:05:14,7227
31,0,A080002005,A080000002,0,2020-04-06 00:38:01,1396
39,0,A060009007,A060000009,0,2020-04-07 01:47:27,7230
44,0,A060016005,A060000016,0,2020-04-16 03:08:13,587
...,...,...,...,...,...,...
260093,7417,A010107006,A010000107,0,2020-08-24 01:26:11,6648
260102,7439,A040003005,A040000003,0,2020-01-14 08:39:48,7268
260106,7439,A040197004,A040000197,0,2020-08-21 07:39:01,2132
260107,7439,A040197005,A040000197,0,2020-08-21 07:39:40,2132


In [16]:
neg_no_valid_inference

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
260104,7439,A040197002,A040000197,1,2020-08-21 07:35:17,2132
260105,7439,A040197003,A040000197,1,2020-08-21 07:36:11,2132
260108,7439,A040197006,A040000197,1,2020-08-21 07:39:45,2132
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832


In [17]:
df_train

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836


In [18]:
test_noinference_data

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623
...,...,...,...,...,...,...
260107,7439,A040197005,A040000197,0,2020-08-21 07:39:40,2132
260108,7439,A040197006,A040000197,1,2020-08-21 07:39:45,2132
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832


In [19]:
with open('neg_no_valid_inference.txt','w',encoding='UTF-8') as f:
    for code,name in neg_no_valid_inference_dic.items():
        f.write(f'{code}')
        for i in range(len(name)):
            f.write(f' {name[i]}')
        f.write(f'\n')

with open('train_no_valid_inference_data.txt','w',encoding='UTF-8') as f:
    for code,name in train_no_valid_inference_data_dic.items():
        f.write(f'{code}')
        for i in range(len(name)):
            f.write(f' {name[i]}')
        f.write(f'\n')

In [20]:
df_test

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623
...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244


In [21]:
test_valid_data.drop(['testID',Timestamp

SyntaxError: unexpected EOF while parsing (640577010.py, line 1)

In [22]:
user_mem=[]
for index, row in test_valid_data.iterrows():
    user_mem.append([int(row['userID']),row['answerCode']])
user_mem=pd.DataFrame(user_mem)
user_mem.to_csv(os.path.join('vallist.csv'), index=False)

In [23]:
# 임시테스트
a=df_test.groupby('userID').tail(1)
a['assemntid8'] = a['assessmentItemID'].apply(lambda x: x[8])
a['assemntid9'] = a['assessmentItemID'].apply(lambda x: x[9])

/tmp/ipykernel_21997/3128204497.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['assemntid8'] = a['assessmentItemID'].apply(lambda x: x[8])
/tmp/ipykernel_21997/3128204497.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['assemntid9'] = a['assessmentItemID'].apply(lambda x: x[9])


In [ ]:
a

In [ ]:
df_mask=a['assemntid9']=='1'

In [ ]:
filtered_df = a[df_mask]

In [ ]:
filtered_df

In [ ]:
df_mask=a['assemntid9']=='3'

In [ ]:
filtered_df = a[df_mask]

In [ ]:
filtered_df

In [ ]:
testtemp=test_noinference_data.groupby('userID').tail(2)

In [ ]:
testtemp=testtemp.groupby('userID').head(1)

In [178]:
user_mem=[]
for index, row in testtemp.iterrows():
    user_mem.append([int(row['userID']),row['answerCode']])
user_mem=pd.DataFrame(user_mem)
user_mem.to_csv(os.path.join('testtemp.csv'), index=False)